# Assignment 3

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model](https://open.spotify.com/track/4R3AU2pjv8ge2siX1fVbZs?si=b2712f32da0e4358) or a simple [linear regression model](https://scikit-learn.org/stable/modules/linear_model.html)
        - An advanced regressor of your choice (e.g., Random Forest, Neural Network, etc.)

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../data/adult/`, then you can use the code below to load them.

In [119]:
import pandas as pd
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('/Users/rajwindertakk/Desktop/Data_Science/scaling_to_production/05_src/data/fires/forestfires.csv', header = None, names = columns))

# Get X and Y

Create the features data frame and target data.

In [120]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV

columns = [
    'coord_X', 'coord_Y', 'month', 'day', 'FFMC', 'DMC', 'DC', 'ISI', 
    'temp', 'RH', 'wind', 'rain', 'area'
]

forestfires = pd.read_csv('/Users/rajwindertakk/Desktop/Data_Science/scaling_to_production/05_src/data/fires/forestfires.csv', names=columns, header=None, na_values='?')

X = forestfires.drop(columns=['area'])
Y = forestfires['area']

scoring = ['neg_log_loss', 'roc_auc', 'f1', 'accuracy', 'precision', 'recall']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [121]:
# preproc1
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numerical_features = ['coord_X', 'coord_Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain']

categorical_features = ['month', 'day']

preproc1 = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='drop'
)
preproc1

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['coord_X', 'coord_Y', 'FFMC', 'DMC', 'DC',
                                  'ISI', 'temp', 'RH', 'wind', 'rain']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [141]:
# preproc2
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline

numerical_features = ['coord_X', 'coord_Y', 'FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH', 'wind', 'rain']

categorical_features = ['month', 'day']

def log_transform(x):
    return np.log1p(x)

preproc2 = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('scaler', RobustScaler()),
            ('log_transform', FunctionTransformer(func=log_transform, validate=False))
        ]), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='drop'
)
preproc2


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', RobustScaler()),
                                                 ('log_transform',
                                                  FunctionTransformer(func=<function log_transform at 0x7fc6c7262ca0>))]),
                                 ['coord_X', 'coord_Y', 'FFMC', 'DMC', 'DC',
                                  'ISI', 'temp', 'RH', 'wind', 'rain']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['month', 'day'])])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [148]:
# Pipeline A = preproc1 + baseline

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

pipeline_A = Pipeline(steps=[('preprocessor', preproc1), ('regressor', LinearRegression())])
cv_results_A = cross_validate(pipeline_A, X, Y, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
cv_results_A

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
   

{'fit_time': array([0.02180982, 0.0119071 , 0.00960302, 0.00964212, 0.00920272]),
 'score_time': array([0.01294994, 0.        , 0.        , 0.        , 0.        ]),
 'test_score': array([nan, nan, nan, nan, nan]),
 'train_score': array([-4771.29921642,            nan,            nan,            nan,
                   nan])}

In [149]:
# Pipeline B = preproc2 + baseline 

pipeline_B = Pipeline(steps=[('preprocessor', preproc2), ('regressor', LinearRegression())])
cv_results_B = cross_validate(pipeline_B, X, Y, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
cv_results_B

/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: divide by zero encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: invalid value encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: divide by zero encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: invalid value encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: divide by zero encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: invalid value encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/371923405

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/pipeline.py", line 475, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/base.py", line 1474, in wrapper
    r

In [125]:
# Pipeline C = preproc1 + advanced model #randomforestregressor(2)

from sklearn.ensemble import RandomForestRegressor
pipeline_C = Pipeline(steps=[('preprocessor', preproc1), ('regressor', RandomForestRegressor())])
cv_results_C = cross_validate(pipeline_C, X, Y, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
cv_results_C

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 345, in _score
    y_pred = method_caller(
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 87, in _cached_call
    result, _ = _get_response_values(
  File "/opt/min

{'fit_time': array([0.46006298, 0.00554013, 0.00458002, 0.00359821, 0.0054481 ]),
 'score_time': array([0.00420594, 0.        , 0.        , 0.        , 0.        ]),
 'test_score': array([nan, nan, nan, nan, nan]),
 'train_score': array([-959.89548053,           nan,           nan,           nan,
                  nan])}

In [150]:
# Pipeline D = preproc2 + advanced model

pipeline_D = Pipeline(steps=[('preprocessor', preproc2), ('regressor', RandomForestRegressor())])
cv_results_D = cross_validate(pipeline_D, X, Y, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
cv_results_D

/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: divide by zero encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: invalid value encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: divide by zero encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: invalid value encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: divide by zero encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3719234055.py:11: RuntimeWarning: invalid value encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/371923405

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/pipeline.py", line 475, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/base.py", line 1474, in wrapper
    r

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [135]:
#Pipeline A

from sklearn.model_selection import GridSearchCV

param_grid = {
    'regressor__fit_intercept': [True, False],
    'regressor__copy_X': [True, False]
}

grid_cv_a = GridSearchCV(
    estimator=pipeline_A,
    param_grid=param_grid,
    cv=5,
    return_train_score=True
)

grid_cv_a.fit(X_train, Y_train)

print("Best parameters found: ", grid_cv_a.best_params_)
print("Best CV score: ", grid_cv_a.best_score_)

Best parameters found:  {'regressor__copy_X': True, 'regressor__fit_intercept': True}
Best CV score:  -0.6592401805971979


In [133]:
#Pipeline B

param_grid = {
    'regressor__fit_intercept': [True, False],
    'regressor__copy_X': [True, False]
}

grid_cv_b = GridSearchCV(
    estimator=pipeline_B,
    param_grid=param_grid,
    cv=5,
    return_train_score=True
)

grid_cv_b.fit(X_train, Y_train)

print("Best parameters found: ", grid_cv_b.best_params_)
print("Best CV score: ", grid_cv_b.best_score_)

/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3951775161.py:11: RuntimeWarning: divide by zero encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3951775161.py:11: RuntimeWarning: invalid value encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3951775161.py:11: RuntimeWarning: divide by zero encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3951775161.py:11: RuntimeWarning: invalid value encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3951775161.py:11: RuntimeWarning: divide by zero encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/3951775161.py:11: RuntimeWarning: invalid value encountered in log1p
  return np.log1p(x)
/var/folders/g9/5l4l9dxd5m3bj63wxx15sg280000gn/T/ipykernel_1223/395177516

ValueError: 
All the 20 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/pipeline.py", line 475, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/base.py", line 1474, in wrapper
   

In [136]:
#Pipeline C

param_grid = {
    'regressor__bootstrap': [True, False],
    'regressor__max_depth': [1, 5, 10, 15],
    'regressor__n_estimators': [100, 200, 300, 400, 500]
}

grid_cv_c = GridSearchCV(
    estimator=pipeline_C,
    param_grid=param_grid,
    cv=5,
    return_train_score=True
)

grid_cv_c.fit(X_train, Y_train)

print("Best parameters found: ", grid_cv_c.best_params_)
print("Best CV score: ", grid_cv_c.best_score_)


Best parameters found:  {'regressor__bootstrap': True, 'regressor__max_depth': 1, 'regressor__n_estimators': 100}
Best CV score:  -0.3481825803879208


# Evaluate

+ Which model has the best performance?
I'm unsure which model as the best performance as only models A and C ran. Models B and D are not working. The results for models A and C both indicate nan (not a number)

In [151]:
train_mse_A = -cv_results_A['train_score'].mean()
test_mse_A = -cv_results_A['test_score'].mean()

print("Average Train MSE for Pipeline A:", train_mse_A)
print("Average Test MSE for Pipeline A:", test_mse_A)


Average Train MSE for Pipeline A: nan
Average Test MSE for Pipeline A: nan


In [153]:
# Assuming grid_cv_b is the tuned GridSearchCV object
best_pipeline_C = grid_cv_c.best_estimator_

# Perform cross-validation on best_pipeline_B
cv_results_C = cross_validate(best_pipeline_C, X, Y, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

train_mse_C = -cv_results_C['train_score'].mean()
test_mse_C = -cv_results_C['test_score'].mean()

print("Average Train MSE for Pipeline B:", train_mse_C)
print("Average Test MSE for Pipeline B:", test_mse_C)


Average Train MSE for Pipeline B: nan
Average Test MSE for Pipeline B: nan


/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
  File "/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
   

# Export

+ Save the best performing model to a pickle file.

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

Unable to complete this portion of the assignment as models A and C and providing an output of non and models B and D did not run. 

*(Answer here.)*

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-3`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_3.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.